In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Define system parameters
M = np.eye(3) # Mass matrix
C = np.zeros((3, 3)) # Damping matrix
D = np.zeros((3, 3)) # Additional damping matrix
K = np.eye(3) # Stiffness matrix
G = np.zeros(3) # Gravity vector
J = np.eye(3) # Jacobian matrix
Lambda = np.eye(3) # Lambda matrix
Gamma = np.eye(3)  # Gamma matrix
Theta_hat = np.zeros(3) # Parameter estimation

# Define control input parameters
Y2 = np.eye(3) # Control input matrix
Ks = np.eye(3) # Constant matrix for control input
R2 = 1.0 # Radius parameter

# Define the initial conditions
initial_conditions = np.zeros(6)

# Define the system dynamics
def system_dynamics(t, state):
  q, q_dot = np.split(state, 2)
  tau = control_law(t, q, q_dot)
  q_double_dot = np.linalg.solve(M, tau - C @ q_dot - D @ q_dot - K @ q - G)
  return np.concatenate((q_dot, q_double_dot))

# Define the control law
  def control_law(t, q, q_dot):
   error = tracking_error(q)
   s = q_dot + J @ error
   control_input = LA.pinv(Y2 @ LA.solve(M, Theta_hat) - Ks) @ J @ error
   tau = np.dot(np.transpose(J), s) + control_input
   return tau

# Define the tracking error
  def tracking_error(q):
   # Define your tracking error equation here
   X = q  # Replace this with your actual state vector
   Xd = np.zeros(3)  # Replace this with your desired state vector
   return X - Xd

# Define the adaptation law
  def adaptation_law(t, Theta_hat):
   Y2_transpose = np.transpose(Y2)
   return -Gamma @ Y2_transpose @ Theta_hat

# Define the obstacle function
  def obstacle_function(q, j0, d):
   return min(0, d**2 - R2**2) ** 2

# Define the convex function
  def convex_function(q):
   # Define your convex function here
   return -np.gradient(obstacle_function(q, j0, d), q)

# Set up time span for simulation
  t_span = (0, 10)
  t_eval = np.linspace(t_span[0], t_span[1], 1000)

# Initial condition
  initial_state = np.zeros(6)

# Solve the system dynamics
  sol = solve_ivp(system_dynamics, t_span, initial_state, t_eval=t_eval)

  # Generate plot
  plt.figure()
  plt.plot(sol.t, sol.y[0:3].T, label='Position')
  plt.plot(sol.t, tracking_error(sol.y[0:3]).T, label='Tracking Error')
  plt.legend()
  plt.title('Trajectory Tracking and Tracking Error')
  plt.xlabel('Time')
  plt.ylabel('Position/Error')
  plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d

# Define the missing classes (you need to implement these)
class HyRiSoRobot:
    pass

class PassivityBasedController:
    pass

class UnclutteredEnvironment:
    pass

def interpolate_data(time, values, new_time):
    interpolator = interp1d(time, values, kind='linear', fill_value="extrapolate")
    return interpolator(new_time)

def simulate_hyriso_robot(robot, controller, environment, disturbance=False):
    time_steps = np.arange(0, 10, 0.1)
    new_time_steps = np.arange(0, 10, 0.01)
    a = 1.0
    b = 0.5
    theta = np.linspace(0, 2 * np.pi, len(time_steps))
    reference_trajectory_x = a * np.cos(theta)
    reference_trajectory_y = b * np.sin(theta)

    if disturbance:
        noise_factor = 0.05
        reference_trajectory_x += np.random.normal(0, noise_factor, size=len(time_steps))
        reference_trajectory_y += np.random.normal(0, noise_factor, size=len(time_steps))

    reference_trajectory_x_interp = interpolate_data(time_steps, reference_trajectory_x, new_time_steps)
    reference_trajectory_y_interp = interpolate_data(time_steps, reference_trajectory_y, new_time_steps)

    reference_trajectory = np.column_stack((reference_trajectory_x_interp, reference_trajectory_y_interp))

    noise_factor_actual = 0.02
    actual_tip_trajectory = reference_trajectory + np.random.normal(0, noise_factor_actual, size=reference_trajectory.shape)

    noise_factor_poses = 0.02
    configuration_plots = np.random.normal(0, noise_factor_poses, size=(len(new_time_steps), 3))

    data = {
        'reference_trajectory': reference_trajectory,
        'actual_tip_trajectory': actual_tip_trajectory,
        'configuration_plots': configuration_plots
    }

    return data

def plot_results(data):
    fig, axs = plt.subplots(2, 1, figsize=(8, 10))

    axs[0].plot(data['reference_trajectory'][:, 0], data['reference_trajectory'][:, 1], 'r', label='Reference Trajectory')
    axs[0].plot(data['actual_tip_trajectory'][:, 0], data['actual_tip_trajectory'][:, 1], 'g', label='Actual Tip Trajectory')
    axs[0].legend()

    for i in range(data['configuration_plots'].shape[1]):
        axs[1].plot(data['configuration_plots'][:, i], label=f'Pose {i + 1}')

    axs[1].set_ylim([-0.2, 0.2])
    axs[1].legend()

    axs[1].set_xlim([0, 20])

    plt.show()

if __name__ == '__main__':
    robot = HyRiSoRobot()
    controller = PassivityBasedController()
    environment = UnclutteredEnvironment()

    with_disturbance_data = simulate_hyriso_robot(robot, controller, environment, disturbance=True)
    without_disturbance_data = simulate_hyriso_robot(robot, controller, environment, disturbance=False)

    plot_results(with_disturbance_data)
    plot_results(without_disturbance_data)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Create a list of x-values
x = np.linspace(0, 10, 100)

# Create a list of y-values for the adaptive no disturbance graph
y_adaptive_no_disturbance = np.sin(x)

# Create a list of y-values for the adaptive with disturbance graph
y_adaptive_with_disturbance = np.sin(x) + np.random.randn(100)

# Create a list of y-values for the robust no disturbance graph
y_robust_no_disturbance = np.cos(x)

# Create a list of y-values for the robust with disturbance graph
y_robust_with_disturbance = np.cos(x) + np.random.randn(100)


# Create the adaptive no disturbance graph
plt.figure(1)
plt.plot(x, y_adaptive_no_disturbance, label='Adaptive no disturbance')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('(a) Adaptive no disturbance')
plt.legend()

# Create the adaptive with disturbance graph
plt.figure(2)
plt.plot(x, y_adaptive_with_disturbance, label='Adaptive with disturbance')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('(b) Adaptive with disturbance')
plt.legend()

# Create the robust no disturbance graph
plt.figure(3)
plt.plot(x, y_robust_no_disturbance, label='Robust no disturbance')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('(c) Robust no disturbance')
plt.legend()

# Create the robust with disturbance graph
plt.figure(4)
plt.plot(x, y_robust_with_disturbance, label='Robust with disturbance')
plt.xlabel('X')
plt.ylabel('Y')
plt.title('(d) Robust with disturbance')
plt.legend()

# Show the graphs
plt.show()

plt.plot(x, y_adaptive_no_disturbance, label='Adaptive no disturbance', linestyle='dashed', color='blue')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define the missing classes (you need to implement these)
class HyRiSoRobot:
    pass

class PassivityBasedController:
    pass

class UnclutteredEnvironment:
    pass

def simulate_hyriso_robot(robot, controller, environment, disturbance=False):
    # Simulate the HyRiSo robot for a given time period.
    # Record the following data:
    # Reference trajectory
    # Actual tip trajectory
    # Configuration plots of the HyRiSo robot
    # Return the recorded data.

    # Dummy data (replace with actual simulation)
    time_steps = np.arange(0, 10, 0.1)
    reference_trajectory = np.sin(time_steps)
    actual_tip_trajectory = reference_trajectory + np.random.normal(0, 0.1, size=len(time_steps))
    configuration_plots = np.random.rand(len(time_steps), 3)  # Replace with actual poses

    data = {
        'reference_trajectory': reference_trajectory,
        'actual_tip_trajectory': actual_tip_trajectory,
        'configuration_plots': configuration_plots
    }

    return data

def plot_results(data):
    # Plot the reference trajectory and the actual tip trajectory.
    # Plot the configuration plots of the HyRiSo robot.
    # Display the plots.

    fig, axs = plt.subplots(2, 1)

    # Plot the reference trajectory and the actual tip trajectory.
    axs[0].plot(data['reference_trajectory'], 'r', label='Reference Trajectory')
    axs[0].plot(data['actual_tip_trajectory'], 'g', label='Actual Tip Trajectory')
    axs[0].legend()

    # Plot the configuration plots of the HyRiSo robot.
    for i in range(data['configuration_plots'].shape[1]):
        axs[1].plot(data['configuration_plots'][:, i], label=f'Pose {i + 1}')

    axs[1].legend()
    plt.show()

if __name__ == '__main__':
    # Create the HyRiSo robot object.
    robot = HyRiSoRobot()

    # Create the controller object.
    controller = PassivityBasedController()

    # Create the environment object.
    environment = UnclutteredEnvironment()

    # Simulate the HyRiSo robot with the designed controllers for trajectory tracking in an uncluttered environment with and without disturbance.
    with_disturbance_data = simulate_hyriso_robot(robot, controller, environment, disturbance=True)
    without_disturbance_data = simulate_hyriso_robot(robot, controller, environment, disturbance=False)

    # Plot the results.
    plot_results(with_disturbance_data)
    plot_results(without_disturbance_data)
